# Temporary code for processing k-means only

In [1]:
%matplotlib inline
#import cython
#%load_ext Cython

In [2]:
import gc
import resource

def mem():
    print('Memory usage         : % 2.2f MB' % round(
        resource.getrusage(resource.RUSAGE_SELF).ru_maxrss/1024.0,1)
    )

mem()

Memory usage         :  64.70 MB


In [3]:
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
import os

import gzip
import sys
import csv

figsize_base = (6.0, 4.0)
plt.rcParams['figure.figsize'] = (6.0, 4.0)
base_image_path = "figures"

def save_fig(name, extension="pdf", add_param={}):
    plt.savefig(os.path.join(base_image_path, "{}.{}".format(name, extension)), bbox_inches="tight", **add_param)


In [4]:
#tracesPath='/var/tmp/clusterdata-2011-2'
tracesPath='/mnt/stockage/leo/google-traces/clusterdata-2011-2'
tasksNums = range(0,500)
tasksNums = range(0,500)

tasksFiles=map(lambda x:tracesPath+'/task_events/part-'+str(x).zfill(5)+'-of-00500.csv.gz', tasksNums)

print tasksFiles[0]
print 'to'
print tasksFiles[-1]

/mnt/stockage/leo/google-traces/clusterdata-2011-2/task_events/part-00000-of-00500.csv.gz
to
/mnt/stockage/leo/google-traces/clusterdata-2011-2/task_events/part-00499-of-00500.csv.gz


In [5]:
preprocessing_file='/mnt/stockage/leo/google-traces/preprocessed.csv'

In [6]:


def read_tasks(tasksFiles, outfile=None):
    tasks={}
    bad_tasks=[]
    arrival=[]
    current_time=0

    complete_tasks_good = []
    complete_tasks_bad = []
    
    outcsv = None
    if outfile is not None:
        outcsv = csv.writer(outfile, delimiter=',')
    
    def make_task_complete(jID, tID):
        (sc, prio, evts) = tasks[(jID, tID)]
        
        if len(evts) < 3:
            #bad_tasks.append((jID, tID))
            pass
        else:
            # XXX suppose the 2nd event is always 'schedule'
            subtime=evts[0][1]
            runtime=evts[1][1]
            endtime=evts[-1][1]

            diff=runtime-subtime
            mt=float(endtime-runtime)/1000000.0
            
            if outcsv is not None:
                outcsv.writerow([evts[-1][0], subtime, sc, prio, diff/1000000.0, 0, mt])
            else:
                toAdd = (sc, prio, diff/1000000.0, 0, mt)
                if evts[-1][0] == 4:
                    complete_tasks_good.append(toAdd)
                else:
                    complete_tasks_bad.append(toAdd)

            del tasks[(jID, tID)]
        
    
    for f in tasksFiles:
        print 'File: ', f
        with gzip.open(f, 'rb') as f:
            for line in f.readlines():
                (timestamp,_,jID,tID,_,evtType,_,schedClass,prio,_,_,_,_)=line.split(',')
                try:
                    if(evtType == '0'):
                        timestamp=int(timestamp)
                        tasks[(int(jID), int(tID))] = (int(schedClass), int(prio), [(0,int(timestamp))])
                        if timestamp>0:
                            arrival.append(timestamp-current_time)
                            current_time=timestamp
                    else:
                        timestamp, jID, tID, evtType = int(timestamp), int(jID), int(tID), int(evtType)
                        (_,_, events)=tasks[(jID,tID)]
                        events.append((evtType,timestamp))

                        if evtType == 4 or evtType == 5:
                            make_task_complete(jID, tID)
                except:
                    bad_tasks.append((int(jID), int(tID)))
            print "Invalid ", len(bad_tasks),
            print "Partial ", len(tasks)
            print "Done : (Finished)", len(complete_tasks_good), "(Killed)", len(complete_tasks_bad)
            mem()
    return arrival,tasks, {'Good': complete_tasks_good, 'Bad': complete_tasks_bad}

In [26]:
def read_from_preprocessed(filename, limit=None):
    tasks_finished = []
    tasks_killed = []
    arrival = []
    
    count = 0
    
    fieldformat = [int, int, int, int, float, float, float]
    row2data = lambda row : (f(v) for f, v in zip(fieldformat, row))
    
    with open(filename, 'r') as f:
        csvfile = csv.reader(f, delimiter=',')
        for row in csvfile:
            lastevt, subtime, sclass, prio, diff, diffrel, mt = row2data(row)
            
            arrival.append(subtime)
            toadd = (sclass, prio, diff, diffrel, mt)
            if lastevt == 4:
                tasks_finished.append(toadd)
            elif lastevt == 5:
                tasks_killed.append(toadd)
                
            count += 1
            if limit is not None and count > limit:
                break

    return {'Good' : tasks_finished, 'Bad' : tasks_killed}, arrival
            

In [7]:
#_, _, tasks = read_tasks(tasksFiles)

In [8]:
mem()
gc.collect()
mem()

Memory usage         :  65.00 MB
Memory usage         :  65.00 MB


In [9]:
if os.path.exists(preprocessing_file):
    print 'ERROR: please remove {} manually (no overwrite)'.format(preprocessing_file)
else:
    with open(preprocessing_file, 'w') as f:
        _, temp_tasks, _ = read_tasks(tasksFiles, f)

File:  /mnt/stockage/leo/google-traces/clusterdata-2011-2/task_events/part-00000-of-00500.csv.gz
Invalid  0 Partial  142915
Done : (Finished) 0 (Killed) 0
Memory usage         :  219.60 MB
File:  /mnt/stockage/leo/google-traces/clusterdata-2011-2/task_events/part-00001-of-00500.csv.gz
Invalid  0 Partial  143084
Done : (Finished) 0 (Killed) 0
Memory usage         :  219.60 MB
File:  /mnt/stockage/leo/google-traces/clusterdata-2011-2/task_events/part-00002-of-00500.csv.gz
Invalid  0 Partial  148847
Done : (Finished) 0 (Killed) 0
Memory usage         :  219.60 MB
File:  /mnt/stockage/leo/google-traces/clusterdata-2011-2/task_events/part-00003-of-00500.csv.gz
Invalid  0 Partial  153148
Done : (Finished) 0 (Killed) 0
Memory usage         :  219.60 MB
File:  /mnt/stockage/leo/google-traces/clusterdata-2011-2/task_events/part-00004-of-00500.csv.gz
Invalid  0 Partial  148991
Done : (Finished) 0 (Killed) 0
Memory usage         :  219.60 MB
File:  /mnt/stockage/leo/google-traces/clusterdata-2011

In [12]:
mem()
del temp_tasks
gc.collect()
mem()

Memory usage         :  1337.50 MB


NameError: name 'temp_tasks' is not defined

In [34]:
classif, arrival = read_from_preprocessed(preprocessing_file, 1000*1000*5)
mem()

Memory usage         :  2027.40 MB


In [35]:
print('Memory for classification : {} MB'.format(sys.getsizeof(classif['Good']) / 1024.0 / 1024.0))
(len(classif['Good']), len(classif['Bad']))

Memory for classification : 26.9358596802 MB


(3349912, 1650089)

# Real code to backport after...
K-means analysis...